In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc, os



from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

In [2]:
#Feature Selection

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import seaborn as sns
from sklearn import metrics

In [3]:
## Import all quarter data
q1_2017 = pd.read_csv('C:/DPA Project/data/Divvy_Trips_2017_Q1.csv')
q2_2017 = pd.read_csv('C:/DPA Project/data/Divvy_Trips_2017_Q2.csv')
q3_2017 = pd.read_csv('C:/DPA Project/data/Divvy_Trips_2017_Q3.csv')
q4_2017 = pd.read_csv('C:/DPA Project/data/Divvy_Trips_2017_Q4.csv')

q1_2018 = pd.read_csv('C:/DPA Project/data/Divvy_Trips_2018_Q1.csv')
q2_2018 = pd.read_csv('C:/DPA Project/data/Divvy_Trips_2018_Q2.csv')
q3_2018 = pd.read_csv('C:/DPA Project/data/Divvy_Trips_2018_Q3.csv')
q4_2018 = pd.read_csv('C:/DPA Project/data/Divvy_Trips_2018_Q4.csv')

q1_2019 = pd.read_csv('C:/DPA Project/data/Divvy_Trips_2019_Q1.csv')
q2_2019 = pd.read_csv('C:/DPA Project/data/Divvy_Trips_2019_Q2.csv')
q3_2019 = pd.read_csv('C:/DPA Project/data/Divvy_Trips_2019_Q3.csv')
q4_2019 = pd.read_csv('C:/DPA Project/data/Divvy_Trips_2019_Q4.csv')


In [4]:
#Station Data

station_data = pd.read_csv("C:/DPA Project/data/Divvy_Bicycle_Stations.csv")
station_data.head()

,ID,Station Name,Total Docks,Docks in Service,Status,Latitude,Longitude,Location
0,675,HQ QR,7,7,In Service,41.889914,-87.680343,"(41.88991358344108, -87.68034316599368)"
1,418,Ellis Ave & 53rd St,11,11,In Service,41.799336,-87.600958,"(41.79933626261, -87.6009581145)"
2,249,Montrose Harbor,31,31,In Service,41.963982,-87.638181,"(41.963982, -87.638181)"
3,322,Kimbark Ave & 53rd St,19,19,In Service,41.799568,-87.594747,"(41.799568, -87.594747)"
4,168,Michigan Ave & 14th St,19,19,In Service,41.864059,-87.623727,"(41.864059, -87.623727)"


In [5]:
## Rename q1_2018 & q2_2019 columns to match others
q1_2018.rename(columns={'01 - Rental Details Rental ID':'trip_id', '01 - Rental Details Local Start Time':'start_time','01 - Rental Details Local End Time':'end_time','01 - Rental Details Bike ID':'bikeid','01 - Rental Details Duration In Seconds Uncapped':'tripduration','03 - Rental Start Station ID':'from_station_id', '03 - Rental Start Station Name':'from_station_name','02 - Rental End Station ID':'to_station_id','02 - Rental End Station Name':'to_station_name', 'User Type':'usertype', 'Member Gender':'gender','05 - Member Details Member Birthday Year':'birthyear'}, inplace=True)

q2_2019.rename(columns={'01 - Rental Details Rental ID':'trip_id', '01 - Rental Details Local Start Time':'start_time','01 - Rental Details Local End Time':'end_time','01 - Rental Details Bike ID':'bikeid','01 - Rental Details Duration In Seconds Uncapped':'tripduration','03 - Rental Start Station ID':'from_station_id', '03 - Rental Start Station Name':'from_station_name','02 - Rental End Station ID':'to_station_id','02 - Rental End Station Name':'to_station_name', 'User Type':'usertype', 'Member Gender':'gender','05 - Member Details Member Birthday Year':'birthyear'}, inplace=True)


In [6]:
## Concatenate all 3 years of data
df = pd.concat([q1_2017, q2_2017, q3_2017, q4_2017,q1_2018, q2_2018, q3_2018, q4_2018, q1_2019,q2_2019,q3_2019,q4_2019])

In [7]:
# Function to calculate missing values by column
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(4)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [8]:
missing_values_table(df)

Your selected dataframe has 12 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
gender,1958538,17.4091
birthyear,1930718,17.1618


In [9]:
df.shape

(11250100, 12)

In [10]:
#Merging Total Docks, To Latitude & To Longitude Data in Dataframe
df['total_docks'] =  df['from_station_id'].map(station_data.set_index('ID')['Total Docks'])
df['to_latitude'] = df['to_station_id'].map(station_data.set_index('ID')['Latitude'])
df['to_longitude'] = df['to_station_id'].map(station_data.set_index('ID')['Longitude'])
df['from_latitude'] = df['from_station_id'].map(station_data.set_index('ID')['Latitude'])
df['from_longitude'] = df['from_station_id'].map(station_data.set_index('ID')['Longitude'])

In [11]:
##Calculating Distnace between origin and destination using lat long

from pyproj import Geod

wgs84_geod = Geod(ellps='WGS84') #Distance will be measured on this ellipsoid - more accurate than a spherical method

#Get distance between pairs of lat-lon points
def Distance(lat1,lon1,lat2,lon2):
  az12,az21,dist = wgs84_geod.inv(lon1,lat1,lon2,lat2) #Yes, this order is correct
  return dist
df['dist_mts'] = Distance(df['to_latitude'].tolist(),df['to_longitude'].tolist(),df['from_latitude'].tolist(),df['from_longitude'].tolist())

In [12]:
#Correcting the data type of tripduration field testing dataset
df['tripduration'] = df['tripduration'].str.replace(',', '')
df['tripduration'] = df['tripduration'].apply(lambda x:float(x))

In [13]:
df['tripduration'] = pd.to_numeric(df['tripduration'], errors='coerce')

In [14]:
#Adding revenue column basis tripduration charging $3/ 30 minutes ride
df['revenue'] = (df['tripduration']/1800) * 3

In [15]:
df['from_station_id'].describe()

count    1.125010e+07
mean     1.904959e+02
std      1.437865e+02
min      1.000000e+00
25%      7.600000e+01
50%      1.650000e+02
75%      2.840000e+02
max      6.730000e+02
Name: from_station_id, dtype: float64

In [16]:
## Find the number of check-outs per station
station_counts_from = pd.DataFrame(df['from_station_id'].value_counts())

In [17]:
## Find the number of check-ins per station
station_counts_to= pd.DataFrame(df['to_station_id'].value_counts())

In [18]:
## Create a DataFrame with the check-outs and check-ins
## Create a columns that sums check-outs and check-ins
station_counts = pd.concat([station_counts_from, station_counts_to], axis=1)
station_counts.rename(columns={'from_station_id':'checkouts', 'to_station_id':'checkins'}, inplace=True)
station_counts['total'] = station_counts['checkouts'] + station_counts['checkins']

In [19]:
## After some research, we will use the top stations with checkin+checkout >50,000
## Stations have a total > 50,000
station_counts_50k = station_counts[station_counts.total>50000]
stations = list(station_counts_50k.index)

In [37]:
## Extract just trips that involve the stations with total checkin checkout > 50K in the model
q2_stations = df[(df['from_station_id'].isin(stations)) | (df['to_station_id'].isin(stations))]

In [39]:
## Convert the 'start_time' column to a datetime object
q2_stations['start_time'] = pd.to_datetime(q2_stations['start_time'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
# drop all rows with any NaN and NaT values
#q2_stations = q2_stations.dropna()

In [44]:
q2_stations['trips'] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
q2_stations['year'] = pd.DatetimeIndex(q2_stations['start_time']).year

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [43]:
q2_stations['month'] = pd.DatetimeIndex(q2_stations['start_time']).month

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
q2_stations['week'] = pd.DatetimeIndex(q2_stations['start_time']).week

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [46]:
q2_stations['weekday'] = q2_stations['start_time'].apply(lambda x: x.strftime("%A"))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [47]:
q2_stations['week_number_of_year'] = q2_stations['start_time'].dt.week

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
q2_stations['hour'] = pd.DatetimeIndex(q2_stations['start_time']).hour

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [50]:
q2_stations.to_csv('C:/DPA Project/result1.csv')

In [49]:
q2_stations.dtypes

trip_id                         int64
start_time             datetime64[ns]
end_time                       object
bikeid                          int64
tripduration                  float64
from_station_id                 int64
from_station_name              object
to_station_id                   int64
to_station_name                object
usertype                       object
gender                         object
birthyear                     float64
total_docks                   float64
to_latitude                   float64
to_longitude                  float64
from_latitude                 float64
from_longitude                float64
dist_mts                      float64
revenue                       float64
year                            int64
month                           int64
trips                           int64
week                            int64
weekday                        object
week_number_of_year             int64
hour                            int64
dtype: objec